In [ ]:
import pandas as pd

#reading in the data
df = pd.read_csv('train2.csv')
test_data = pd.read_csv('test.csv')
# df = df[0:100000]
# test_data = test_data[0:10000]
# print(df)
# df.head()

from io import StringIO
col = ['Category_name', 'title', 'Category']
df = df[col]
df = df[pd.notnull(df['title'])]
df.columns = ['Category_name', 'title', 'Category']

# df['Category'] = df['Category']
category_id_df = df[['Category_name', 'Category']].drop_duplicates().sort_values('Category')
category_to_id = dict(category_id_df.values)
id_to_category = dict(category_id_df[['Category', 'Category_name']].values)
df.head()


from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(sublinear_tf=True, min_df=1, norm='l2', encoding='latin-1', ngram_range=(1, 1), stop_words='english')

# training set
features = tfidf.fit_transform(df.title).toarray()
print('features', features)
labels = df.Category

#testing set
test_features = tfidf.fit_transform(test_data.title).toarray()
print('test_features', test_features)

features.shape

from sklearn.feature_selection import chi2
import numpy as np
N = 2


for Category_name, Category in sorted(category_to_id.items()):
  features_chi2 = chi2(features, labels == Category)
  indices = np.argsort(features_chi2[0])
  feature_names = np.array(tfidf.get_feature_names())[indices]
  unigrams = [v for v in feature_names if len(v.split(' ')) == 1]
  bigrams = [v for v in feature_names if len(v.split(' ')) == 2]
#   print("# '{}':".format(Category))
#   print("  . Most correlated unigrams:\n. {}".format('\n. '.join(unigrams[-N:])))
#   print("  . Most correlated bigrams:\n. {}".format('\n. '.join(bigrams[-N:])))
    
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB

X_train, X_test, y_train, y_test = train_test_split(df['title'], df['Category_name'], random_state = 0)
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
clf = MultinomialNB().fit(X_train_tfidf, y_train)
    
# print(clf.predict(count_vect.transform(["apple phone"])))

import matplotlib.pyplot as plt


from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score

models = [
    RandomForestClassifier(n_estimators=200, max_depth=3, random_state=0),
    LinearSVC(),
    MultinomialNB(),
    LogisticRegression(random_state=0),
]

CV = 5
cv_df = pd.DataFrame(index=range(CV * len(models)))
entries = []

for model in models:
  model_name = model.__class__.__name__
  accuracies = cross_val_score(model, features, labels, scoring='accuracy', cv=CV)
  for fold_idx, accuracy in enumerate(accuracies):
    entries.append((model_name, fold_idx, accuracy))
cv_df = pd.DataFrame(entries, columns=['model_name', 'fold_idx', 'accuracy'])

import seaborn as sns

sns.boxplot(x='model_name', y='accuracy', data=cv_df)
sns.stripplot(x='model_name', y='accuracy', data=cv_df, 
              size=8, jitter=True, edgecolor="gray", linewidth=2)
plt.show()

cv_df.groupby('model_name').accuracy.mean()

model = LinearSVC()
# print('df.index', df.index)
# X_train, X_test, y_train, y_test, indices_train, indices_test = train_test_split(features, labels, df.index, test_size=0.33, random_state=0)

X_train = features
y_train = labels
X_test = test_features

print('X_train', X_train)
print('y_train', y_train)
print('X_test', X_test)

model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print('y_pred', y_pred)

from sklearn.metrics import confusion_matrix

conf_mat = confusion_matrix(y_test, y_pred)
fig, ax = plt.subplots(figsize=(10,10))
sns.heatmap(conf_mat, annot=True, fmt='d',
            xticklabels=category_id_df.Category_name.values, yticklabels=category_id_df.Category_name.values)
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.show()

from IPython.display import display

# print(category_id_df)

# for predicted in category_id_df.Category:
#     for actual in category_id_df.Category:
# #         print('predicted', predicted)
# #         print('actual', actual)
#         if predicted != actual and conf_mat[actual, predicted] >= 10:
#           print("'{}' predicted as '{}' : {} examples.".format(id_to_category[actual], id_to_category[predicted], conf_mat[actual, predicted]))
#           display(df.loc[indices_test[(y_test == actual) & (y_pred == predicted)]][['Category_name', 'title']])
#           print('')

            
# model.fit(features, labels)

# N = 2
# for Category_name, Category in sorted(category_to_id.items()):
#   indices = np.argsort(model.coef_[Category])
#   feature_names = np.array(tfidf.get_feature_names())[indices]
#   unigrams = [v for v in reversed(feature_names) if len(v.split(' ')) == 1][:N]
#   bigrams = [v for v in reversed(feature_names) if len(v.split(' ')) == 2][:N]
#   print("# '{}':".format(Category_name))
#   print("  . Top unigrams:\n       . {}".format('\n       . '.join(unigrams)))
#   print("  . Top bigrams:\n       . {}".format('\n       . '.join(bigrams)))


# from sklearn import metrics

# print(metrics.classification_report(y_test, y_pred, target_names=df['Category_name'].unique()))



features [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
test_features [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
